In [1]:
!pip install git+https://github.com/WildlifeDatasets/wildlife-datasets@develop
!pip install git+https://github.com/WildlifeDatasets/wildlife-tools

  Cloning https://github.com/WildlifeDatasets/wildlife-datasets (to revision develop) to /tmp/pip-req-build-6xnyvq3z
  Running command git clone --filter=blob:none --quiet https://github.com/WildlifeDatasets/wildlife-datasets /tmp/pip-req-build-6xnyvq3z
  Running command git checkout -b develop --track origin/develop
  Switched to a new branch 'develop'
  Branch 'develop' set up to track remote branch 'develop' from 'origin'.
  Resolved https://github.com/WildlifeDatasets/wildlife-datasets to commit 298b9a0b566a56ccc76727928a1bb8126833730a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wildlife-datasets: filename=wildlife_datasets-1.0.5-py3-none-any.whl size=88495 sha256=ef950ea5eeb25a4d20f33a840e2f91fd24c63b3c04327ca5d3bef6b3995024b0
  Stored in directory: /tmp/pip-ephem-wheel-cache-f9r2a6r6/wheels/c5/e9/19/815dc8ac1a073b6769942e201b8776a681537b3da4bc2c60ab
Successfully built wi

In [9]:
import os
import numpy as np
import pandas as pd
import timm
import torchvision.transforms as T
from wildlife_datasets.datasets import AnimalCLEF2025, WildlifeDataset
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity

In [4]:
root = '/kaggle/input/nto-reid-data/'
# transform_display = T.Compose([
#     T.Resize([256, 240]),
# ])
# transform = T.Compose([
#     *transform_display.transforms,
#     T.ToTensor(),
#     T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# transforms_aliked = T.Compose([
#     T.Resize([512, 512]),
#     T.ToTensor()
# ])

In [5]:
metadata = pd.read_csv(root + 'train.csv')

In [19]:
transform = T.Compose([
    T.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    T.RandAugment(num_ops=2, magnitude=15),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

In [21]:
dataset = WildlifeDataset(root=root + 'train', metadata=metadata, col_label='label', col_path='file_name', load_label=True, transform=transform)

In [11]:
unique_identities = dataset.metadata['label'].unique()

In [12]:
from itertools import chain
import torch
import timm
import pandas as pd
import torchvision.transforms as T
from torch.optim import SGD
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.train import ArcFaceLoss, BasicTrainer

In [13]:
import torch
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus[0]

In [ ]:
backbone = timm.create_model('swin_large_patch4_window7_224', num_classes=0, pretrained=True)
# backbone = timm.create_model('efficientvit_b2', num_classes=0, pretrained=True)
with torch.no_grad():
    dummy_input = torch.randn(1, 3, 224, 224)
    embedding_size = backbone(dummy_input).shape[1]
objective = ArcFaceLoss(num_classes=dataset.num_classes, embedding_size=embedding_size, margin=0.5, scale=64)


# Optimizer and scheduler configuration
params = chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)
min_lr = optimizer.defaults.get("lr") * 1e-3
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=min_lr)
devices = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Setup training
trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    device=0,
    objective=objective,
    optimizer=optimizer,
    scheduler=scheduler,
    batch_size=8,
    accumulation_steps=8,
    num_workers=2,
    epochs=30,
)


trainer.train()

Epoch 2:  39%|████████████████████▌                               | 376/952 [02:06<03:12,  2.99it/s]